In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
import datetime

In [16]:
#DEFINE FUNÇÕES PRINCIPAIS PARA ACESSO DO BANCO DE DADOS COM A BASE INICIAL

conn = sqlite3.connect('DIST_DB_ANEEL.db')
cursor = conn.cursor()
conn_bi = sqlite3.connect('BD_POWER_BI_FINAL.db')
cursor_bi = conn_bi.cursor()

In [13]:
#PREPARA TABELA COM INFORMAÇÕES UTEIS PARA ANALISE

tableName1 = "DIST_RECLAMACOES_N1_N2"
columnsNecessary = f"SigAgente,DatReferencia,DescReclamacao,QtdReclamacoesRecebidas,QtdReclamacoesImprocedentes "

query = f"""
        SELECT {columnsNecessary}
        FROM {tableName1}
        WHERE SigAgente = "ENEL CE"

        """
df = pd.read_sql_query(query,conn)


In [17]:
def removeMotivoInvalido(df): #Remove as reclamaçoes com motivos sem aplicação para esta analise
    df['MotivoValido'] = ''
    listaRemocao = ['Elogio','Informação','Alteração Cadastral','Denúncia','Custo de Disponibilidade','Sugestão','Problemas de instalação interna na unidade consumidora',\
                    'Prazos','Indisponibilidade de Agência/Postos e Canais de atendimento/Atend. Telefônico/Serviço de Arrecadação','Cobrança Indevida de Atividade Acessória']

    df['MotivoValido'] = ~df['DescReclamacao'].isin(listaRemocao)
    df = df[df['MotivoValido']]

    return df

df_analise = df.copy()
df_analise['QtdReclamacoesProcedentes'] = df_analise['QtdReclamacoesRecebidas'] - df_analise['QtdReclamacoesImprocedentes'] #Calcula a diferença entre reclamações recebidas e as improcedentes
df_analise = df_analise.drop(columns=['QtdReclamacoesRecebidas','QtdReclamacoesImprocedentes']) # Deleta colunas não utilizadas
df_analise['Ano'] = df_analise['DatReferencia'].apply(lambda x: x[0:4]) #Cria coluna ano
df_analise['DatReferencia'] = pd.to_datetime(df_analise['DatReferencia']) 
#print(set(df['DescReclamacao']))


df_analise = removeMotivoInvalido(df_analise) #Remove as reclamaçoes com motivos sem aplicação para esta analise

#Agrupa por tipo e soma a quantidade de reclamações - Analise 1
df_analise1 = df_analise.groupby(['DescReclamacao'])['QtdReclamacoesProcedentes'].sum().reset_index().sort_values('QtdReclamacoesProcedentes',ascending=False)

#Agrupa por ano e soma a quantidade de reclamacoes - Analise 2
df_analise2 = df_analise.groupby(['Ano'])['QtdReclamacoesProcedentes'].sum().reset_index().sort_values('Ano',ascending=True)
df_analise2['AumentoPercentual'] = df_analise2['QtdReclamacoesProcedentes'].pct_change()
df_analise2['DatReferencia'] = df_analise2['Ano']
df_analise2['DatReferencia'] = df_analise2['DatReferencia'].apply(lambda x: "01-01-"+str(x))
df_analise2['DatReferencia'] = pd.to_datetime(df_analise2['DatReferencia'])
print(df_analise2)




     Ano  QtdReclamacoesProcedentes  AumentoPercentual DatReferencia
0   2010                     657585                NaN    2010-01-01
1   2011                     744047           0.131484    2011-01-01
2   2012                     657809          -0.115904    2012-01-01
3   2013                     724007           0.100634    2013-01-01
4   2014                     738234           0.019650    2014-01-01
5   2015                     734244          -0.005405    2015-01-01
6   2016                     773372           0.053290    2016-01-01
7   2017                     746250          -0.035070    2017-01-01
8   2018                     758014           0.015764    2018-01-01
9   2019                    1039975           0.371973    2019-01-01
10  2020                    1209168           0.162689    2020-01-01
11  2021                     895688          -0.259253    2021-01-01
12  2022                     986027           0.100860    2022-01-01
13  2023                     46503

In [18]:
#SOBE PARA O BANCO DE DADOS

df_analise1.to_sql('ReclamacoesN1N2_1',conn_bi, if_exists='replace', index=False)
df_analise2.to_sql('ReclamacoesN1N2_2',conn_bi, if_exists='replace', index=False)
df_analise.to_sql('ReclamacoesN1N2_Geral',conn_bi, if_exists='replace', index=False)

conn.close()
conn_bi.close()